In [ ]:
import xarray as xr

import pandas as pd
import numpy as np

I currently seem to have gfs forecasts starting from april till september.

In [ ]:
gfs_location = "/home/eric/data/gfs_forecasts/gfs.0p25.2022080100.f000.grib2"
ds = xr.load_dataset(gfs_location, engine="cfgrib", filter_by_keys={'typeOfLevel': 'isobaricInhPa'})

# Opening multiple gfs files:

In [ ]:
start = "2022-08-01"
end = "2022-08-31"

forecast_days = [day.strftime("%Y%m%d%H") for day in pd.date_range(start=start, end=end, freq="1D")]
base_string = "/home/eric/data/gfs_forecasts/"
skip_day_list = [
    "2022040700",
    "2022061500"
]
multi_filename = []
for day in forecast_days:
    if day in skip_day_list:
        continue
    file_names = []
    for ahead_time in np.arange(0,24, 3):
        file_names.append(base_string + f"gfs.0p25.{day}.f{ahead_time:03d}.grib2")
    multi_filename.append(file_names)

In [ ]:
ds_multi = xr.open_mfdataset(multi_filename, combine='nested', concat_dim=['time', 'step'], filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'isobaricInhPa'})

In [ ]:
ds_multi = ds_multi.rename({'time': 'run_date'})
ds_multi_new = ds_multi.stack(time=['run_date', 'step'])
ds_multi_new = ds_multi_new.drop_vars({'time', 'run_date', 'step'}).assign_coords({"time": ds_multi_new.valid_time})

In [ ]:
ds_multi_new['t'][0, 50, 50, :].plot()

In [ ]:
ds_multi_new.to_netcdf("/home/eric/data/gfs_forecasts_processed/gfs_processed_august.nc")

# Parsing GFS for weather stations

In [ ]:
start = "2022-08-01"
end = "2022-08-31"

forecast_days = [day.strftime("%Y%m%d%H") for day in pd.date_range(start=start, end=end, freq="1D")]
base_string = "/home/eric/data/gfs_forecasts/"
skip_day_list = [
    "2022040700",
    "2022061500"
]
multi_filename = []
for day in forecast_days:
    if day in skip_day_list:
        continue
    file_names = []
    for ahead_time in np.arange(0,24, 3):
        file_names.append(base_string + f"gfs.0p25.{day}.f{ahead_time:03d}.grib2")
    multi_filename.append(file_names)

In [ ]:
ds_multi = xr.open_mfdataset(multi_filename, combine='nested', concat_dim=['time', 'step'], filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'surface'})

In [ ]:
ds_multi = ds_multi.rename({'time': 'run_date'})
ds_multi_new = ds_multi.stack(time=['run_date', 'step'])
ds_multi_new = ds_multi_new.drop_vars({'time', 'run_date', 'step'}).assign_coords({"time": ds_multi_new.valid_time})

In [ ]:
ds_multi_new.to_netcdf("/home/eric/data/gfs_forecasts_processed/gfs_processed_surface_august.nc")

In [ ]:
! ls /home/eric/data/gfs_forecasts_processed/